# Initialization:

In [1]:
import torch
import dqc
import dqc.xc
import dqc.utils

from DQCAdapter import DQCAdapter

In [2]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a_par, p_par):
        super().__init__()
        self.a_par = a_par
        self.p_par = p_par
        self.number_of_parameters = 2

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a_par * rho ** self.p_par
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p_par
            elif number_of_parameter == 1: # parameter p
                return self.a_par * torch.log(rho) * rho ** self.p_par

In [3]:
a_par = torch.nn.Parameter(torch.tensor(1.0, dtype=torch.double))
p_par = torch.nn.Parameter(torch.tensor(2.0, dtype=torch.double))
myxc = MyLDAX(a_par, p_par)

In [4]:
water = """
O        0.000000000      0.000000000      0.000000000;
H        0.000000000      1.434938863      1.126357947;
H        0.000000000     -1.434938863      1.126357947
"""

mol = dqc.Mol(moldesc=water, basis="6-31G")
qc = dqc.KS(mol, xc=myxc).run()
ene = qc.energy()
print(ene)

tensor(-38.2050, dtype=torch.float64, grad_fn=<AddBackward0>)


In [5]:
adapter = DQCAdapter(qc)

In [6]:
fockian = adapter.get_fockian()
print(fockian.size())

torch.Size([13, 13])


In [7]:
coefficients = adapter.get_orbital_coefficients()
coefficients.size()

torch.Size([13, 5])

In [8]:
orbital_energies = adapter.get_orbital_energies()
orbital_energies.size()

torch.Size([5])

In [9]:
occupancy = adapter.get_orbital_occupancy()
occupancy.size()

torch.Size([5])

In [10]:
number_of_occupied_orbitals = adapter.get_number_of_occupied_orbitals()

# Calculating adjoint derivatives:

## Derivatives from energy:

### Calculating $\frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}}$

#### The first way to calculate derivative from energy with respect to coefficients:
Every type of orbitals can be used

$$ \frac{\partial E[\rho](\vec{\theta})}{\partial C_{bj}} = 2f_b \sum_i C_{bi}F_{ij}$$
so
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}} = 2\textbf{f}\textbf{C}\textbf{F}$$
$\textbf{f}$ here is matrix: $f_{ab}=\delta_{ab}f_a$

In [11]:
dE_wrt_dC_first_way = 2 * torch.einsum("b,ib,ij->bj", occupancy, coefficients, fockian)
dE_wrt_dC_first_way.size()

torch.Size([5, 13])

#### The second way to calculate derivative from energy with respect to coefficients
Only canonical (i.e. eigenfunctions of fockian) orbitals can be used

from the other hand, 
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial C_{bj}} = 2f_b \sum_i C_{bi}F_{ij} = 2f_b \epsilon_b C_{bj}$$
so
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}} = 2\textbf{f}\epsilon\textbf{C}$$
$\epsilon$ here is matrix: $\epsilon_{ab}=\delta_{ab}\epsilon_a$

In [12]:
dE_wrt_dC_second_way = 2 * torch.einsum("b,b,jb->bj", occupancy, orbital_energies, coefficients)
dE_wrt_dC_second_way.size()

torch.Size([5, 13])

Check that both ways lead to the same results:

In [13]:
print(torch.linalg.matrix_norm(dE_wrt_dC_first_way - dE_wrt_dC_second_way))

tensor(8.7654e-14, dtype=torch.float64, grad_fn=<CopyBackwards>)


### Calculating $\frac{\partial E[\rho](\vec{\theta})}{\partial \vec{\epsilon}}$

$$\frac{\partial E[\rho](\vec{\theta})}{\partial \epsilon_{b}} = 0$$ for all $\epsilon_b$. So:

In [14]:
dE_wrt_depsilon = torch.zeros(number_of_occupied_orbitals)
dE_wrt_depsilon.size()

torch.Size([5])

## Derivatives from normalization equations:

### Calculating $\frac{\partial \vec{r}(\textbf{C})}{\partial \textbf{C}}$

$$\frac{\partial r_{a}(\textbf{C})}{\partial C_{ck}} = 2\delta_{ac}C_{ck}$$


In [15]:
occupied_orbitals_kronecker = torch.eye(number_of_occupied_orbitals)

In [16]:
dnorm_wrt_dC = 2 * torch.einsum("ac,kc->kac", occupied_orbitals_kronecker, coefficients)
dnorm_wrt_dC.size()

torch.Size([13, 5, 5])

### Calculating $\frac{\partial \vec{r}(\textbf{C})}{\partial \vec{\epsilon}}$

$$\frac{\partial r_{a}(\textbf{C})}{\partial \epsilon_b} = 0$$


In [17]:
dnorm_wrt_depsilon = torch.zeros((number_of_occupied_orbitals, number_of_occupied_orbitals))
dnorm_wrt_depsilon.size()

torch.Size([5, 5])

## Derivatives from Roothan equations:

### Calculating $\frac{\partial \textbf{r}(\textbf{C};\;\vec{\theta})}{\partial \textbf{C}}$

$$G_{Cou, ijkl} = \iint b_i(\vec{r}) \frac{b_k(\vec{r'})b_l(\vec{r'})}{|\vec{r}-\vec{r'}|} b_j(\vec{r})d\vec{r'}d\vec{r}$$
$$G_{XC, ijkl} = 
\int b_i(\vec{r}) b_k(\vec{r}) \frac{\partial V_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \rho(\vec{r})}b_l(\vec{r})b_j(\vec{r})d\vec{r}$$
$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial C_{ck}} = 
(F_{ik}[\rho](\vec{\theta}) - \epsilon_c\delta_{ik})\delta_{ac} +
2f_c\sum_j \sum_{l}C_{cl}\left(G_{Cou,ijkl} + G_{XC,ijkl}\right)C_{aj}
$$

First term:

In [18]:
number_of_all_orbitals = adapter.get_number_of_all_orbitals()
all_orbitals_kronecker = torch.eye(number_of_all_orbitals)
dRoothan_wrt_dC_first_term = torch.einsum("ik,ac->iakc", fockian, occupied_orbitals_kronecker)
dRoothan_wrt_dC_first_term -= torch.einsum("c,ik,ac->iakc", orbital_energies, all_orbitals_kronecker, occupied_orbitals_kronecker)
dRoothan_wrt_dC_first_term.size()

torch.Size([13, 5, 13, 5])

Second term:
(NB: fourDtensor loses symmetry of XC-tensor):

In [19]:
fourDtensor = adapter.get_four_center_elrep_tensor() + adapter.get_four_center_xc_tensor()
dRoothan_wrt_dC_second_term = 2 * torch.einsum("c,ijkl,lc,ja->iakc", occupancy, fourDtensor, coefficients, coefficients)
dRoothan_wrt_dC_second_term.size()

torch.Size([13, 5, 13, 5])

The sum:

In [20]:
dRoothan_wrt_dC = dRoothan_wrt_dC_first_term + dRoothan_wrt_dC_second_term
dRoothan_wrt_dC.size()

torch.Size([13, 5, 13, 5])

### Calculating $\frac{\partial \textbf{r}(\textbf{C};\;\vec{\theta})}{\partial \vec{\epsilon}}$


$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial \epsilon_{c}} = -\delta_{ac}C_{ai}$$

In [21]:
dRoothan_wrt_depsilon = -1 * torch.einsum("ac,ia->iac", occupied_orbitals_kronecker, coefficients)
dRoothan_wrt_depsilon.size()

torch.Size([13, 5, 5])

# Calculate adjoint vector

## Concatenate tensors:

### $\frac{\partial E}{\partial \vec{X}}$

$$\left( \frac{\partial E}{\partial \vec{X}} \right)_{N_{ao}j+c} =  \begin{cases}
   \frac{\partial E}{\partial С_{cj}} &\text{if $0 \le j < N_{orb}$} \\
   \frac{\partial E}{\partial \epsilon_{c}} &\text{if $j = N_{orb}$}
 \end{cases}$$

In [22]:
print(dE_wrt_depsilon.unsqueeze(-1).size())
print(dE_wrt_dC_second_way.size())
dE_wrt_dX = torch.cat((dE_wrt_dC_second_way, dE_wrt_depsilon.unsqueeze(-1)), 1)
print(dE_wrt_dX.size())
dE_wrt_dX = dE_wrt_dX.t().reshape(-1,)
print(dE_wrt_dX.size())

torch.Size([5, 1])
torch.Size([5, 13])
torch.Size([5, 14])
torch.Size([70])


Check it:

In [23]:
for j in range(number_of_all_orbitals + 1):
    for c in range(number_of_occupied_orbitals):
        index = number_of_occupied_orbitals * j + c
        if (j < number_of_all_orbitals):
            assert(dE_wrt_dX[index] == dE_wrt_dC_second_way[c][j])
        else:
            assert(dE_wrt_dX[index] == dE_wrt_depsilon[c])

### $\frac{\partial \vec{Y}}{\partial \vec{X}}$

$$\left( \frac{\partial \vec{Y}}{\partial \vec{X}} \right)_{N_{ao}i+a,N_{ao}j+c} =  \begin{cases}
   \frac{\partial r_{ia}}{\partial С_{cj}} &\text{if $0 \le i < N_{orb}$ and $0 \le j < N_{orb}$} \\
   \frac{\partial r_{ia}}{\partial \epsilon_{c}} &\text{if $0 \le i < N_{orb}$ and $j = N_{orb}$} \\
   \frac{\partial r_{a}}{\partial С_{cj}} &\text{if $i = N_{orb}$ and $0 \le j < N_{orb}$} \\
   \frac{\partial r_{a}}{\partial \epsilon_{c}} &\text{if $i = N_{orb}$ and $j = N_{orb}$}
 \end{cases}$$

In [24]:
nao_norb_product = number_of_all_orbitals * number_of_occupied_orbitals
upper_left = dRoothan_wrt_dC.reshape((nao_norb_product,nao_norb_product))
print(dRoothan_wrt_dC.size(), "=>", upper_left.size())
upper_right = dRoothan_wrt_depsilon.reshape((nao_norb_product, number_of_occupied_orbitals))
print(dRoothan_wrt_depsilon.size(), "=>", upper_right.size())
upper = torch.cat((upper_left, upper_right), 1)
print(upper_left.size(), "+", upper_right.size(), "=", upper.size())

torch.Size([13, 5, 13, 5]) => torch.Size([65, 65])
torch.Size([13, 5, 5]) => torch.Size([65, 5])
torch.Size([65, 65]) + torch.Size([65, 5]) = torch.Size([65, 70])


In [25]:
lower_left = dnorm_wrt_dC.reshape(nao_norb_product, number_of_occupied_orbitals).t()
print(dnorm_wrt_dC.size(), "=>", lower_left.size())
lower = torch.cat((lower_left, dnorm_wrt_depsilon), 1)
print(lower_left.size(), "+", dnorm_wrt_depsilon.size(), "=", lower.size())

torch.Size([13, 5, 5]) => torch.Size([5, 65])
torch.Size([5, 65]) + torch.Size([5, 5]) = torch.Size([5, 70])


In [26]:
dY_wrt_dX = torch.cat((upper, lower), 0)
print(upper.size(), "+", lower.size(), "=", dY_wrt_dX.size())

torch.Size([65, 70]) + torch.Size([5, 70]) = torch.Size([70, 70])


Check it:

In [27]:
for i in range(number_of_all_orbitals + 1):
    for a in range(number_of_occupied_orbitals):
        for j in range(number_of_all_orbitals + 1):
            for c in range(number_of_occupied_orbitals):
                index = (number_of_occupied_orbitals * i + a, number_of_occupied_orbitals * j + c)
                if (i < number_of_all_orbitals):
                    if (j < number_of_all_orbitals):
                        assert(dY_wrt_dX[index[0]][index[1]] == dRoothan_wrt_dC[i][a][j][c]),(i,a,j,c)
                    else:
                        assert(dY_wrt_dX[index[0]][index[1]] == dRoothan_wrt_depsilon[i][a][c]),(i,a,c)
                else:
                    if (j < number_of_all_orbitals):
                        assert(dY_wrt_dX[index[0]][index[1]] == dnorm_wrt_dC[j][a][c]),(j,a,c)
                    else:
                        assert(dY_wrt_dX[index[0]][index[1]] == dnorm_wrt_depsilon[a][c]),(a,c)

## Find inverse matrix of $\frac{\partial \vec{Y}}{\partial \vec{X}}$

In [28]:
dY_wrt_dX_inverse = torch.linalg.inv(dY_wrt_dX)
print(dY_wrt_dX_inverse.size())

torch.Size([70, 70])


In [29]:
dY_wrt_dX_inverse

tensor([[ 1.5837e-01, -7.1862e-03, -8.0783e-03,  ...,  1.1285e-04,
          9.2958e-03,  1.7079e-02],
        [-7.1862e-03,  7.9609e-01, -1.1530e-02,  ...,  1.5451e-03,
          1.7847e-02,  1.2349e-02],
        [-8.0783e-03, -1.1530e-02,  7.4811e-01,  ...,  1.2581e-01,
          1.2147e-01,  6.2812e-02],
        ...,
        [-2.2569e-04, -3.0903e-03, -2.5163e-01,  ...,  9.1648e-01,
          7.9537e-01,  7.3286e-01],
        [-1.8592e-02, -3.5693e-02, -2.4293e-01,  ...,  7.9537e-01,
          1.3765e+00,  8.0069e-01],
        [-3.4157e-02, -2.4698e-02, -1.2562e-01,  ...,  7.3286e-01,
          8.0069e-01,  8.6596e-01]], dtype=torch.float64,
       grad_fn=<LinalgInvExBackward0>)

In [30]:
torch.linalg.matrix_norm((dY_wrt_dX_inverse @ dY_wrt_dX) - torch.eye(dY_wrt_dX.shape[0]))
torch.linalg.matrix_norm((dY_wrt_dX @ dY_wrt_dX_inverse) - torch.eye(dY_wrt_dX.shape[0]))

tensor(5.0420e-13, dtype=torch.float64, grad_fn=<CopyBackwards>)

## Find adjoint vector:

In [31]:
adjoint_vector = torch.matmul(dY_wrt_dX_inverse, dE_wrt_dX)
print(adjoint_vector.size())

torch.Size([70])


# Calculating derivatives with respect to parameters

## Derivative from energy: $\frac{\partial E[\rho](\vec{\theta})}{\partial \vec{\theta}}$

$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \vec{\theta}} = 
 \int  \frac{\partial \rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}}d\vec{r} 
$$
so we just can put $\frac{\partial \rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}}$ instead of $\rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})$ to DQC calculation.

In [32]:
dE_wrt_dtheta = adapter.get_derivative_of_exc_wrt_theta()
print(dE_wrt_dtheta.size())

torch.Size([2])


## Derivative from normalization equations:

$$\frac{\partial r_{a}(\textbf{C})}{\partial\vec{\theta}}= 0$$ so

In [33]:
number_of_parameters = adapter.get_number_of_parameters()
dnorm_wrt_dtheta = torch.zeros(number_of_occupied_orbitals, number_of_parameters)
print(dnorm_wrt_dtheta.size())

torch.Size([5, 2])


## Derivative from Roothan equations: 

$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial \vec{\theta}} =
\sum_j C_{aj}\int b_i(\vec{r}) \frac{\partial V_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}} b_j(\vec{r})d\vec{r}$$

As we know, $$V_{XC}[\rho](\vec{r};\;\vec{\theta}) = \frac{\partial E_{XC}[\rho]}{\partial\rho(\vec{r})} = \frac{\partial \rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial\rho(\vec{r})}$$
So

$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial \vec{\theta}} =
\sum_j C_{aj}\int b_i(\vec{r}) \frac{\partial\rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}\partial\rho(\vec{r})} b_j(\vec{r})d\vec{r} = 
\sum_j C_{aj}\int b_i(\vec{r}) \frac{ \frac{\partial \rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}}}{\partial\rho(\vec{r})} b_j(\vec{r})d\vec{r}
$$
It means, we can use $\frac{\partial \rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}}$ instead of $\rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})$ in DQC function`get_vxc()`and get suitable result. This function takes densinfo and takes functional derivative with respect to density.

In [34]:
dvxc_wrt_dtheta = adapter.get_derivative_of_vxc_wrt_theta()
print(dvxc_wrt_dtheta.size())

torch.Size([13, 13, 2])


In [35]:
dRoothan_wrt_dtheta = torch.einsum("ijt,ja->iat", dvxc_wrt_dtheta, coefficients)
print(dRoothan_wrt_dtheta.size())

torch.Size([13, 5, 2])


# Concatenate derivatives with respect to parameters

In [36]:
dY_wrt_dtheta = dRoothan_wrt_dtheta.reshape(nao_norb_product, number_of_parameters)
print(dRoothan_wrt_dtheta.size(), "=>", dY_wrt_dtheta.size())
dY_wrt_dtheta = torch.cat((dY_wrt_dtheta, dnorm_wrt_dtheta), 0)
print(dY_wrt_dtheta.size())

torch.Size([13, 5, 2]) => torch.Size([65, 2])
torch.Size([70, 2])


Check it:

In [37]:
for t in range(number_of_parameters):
    for i in range(number_of_all_orbitals + 1):
        for a in range(number_of_occupied_orbitals):
            index = number_of_occupied_orbitals * i + a
            if (i < number_of_all_orbitals):
                assert(dY_wrt_dtheta[index][t] == dRoothan_wrt_dtheta[i][a][t]),(i,a,t)
            else:
                assert(dY_wrt_dtheta[index][t] == dnorm_wrt_dtheta[a][t]),(a,t)

# Total derivative:

In [38]:
print("partial derivative:", dE_wrt_dtheta)
print(torch.matmul(adjoint_vector, dY_wrt_dtheta))
total_dE_wrt_dtheta = dE_wrt_dtheta - torch.matmul(adjoint_vector, dY_wrt_dtheta)
print("full derivative:", total_dE_wrt_dtheta)

partial derivative: tensor([ 9.9156, 19.0426], dtype=torch.float64, grad_fn=<CatBackward0>)
tensor([-8.9951e-15, -3.3579e-14], dtype=torch.float64,
       grad_fn=<SqueezeBackward3>)
full derivative: tensor([ 9.9156, 19.0426], dtype=torch.float64, grad_fn=<SubBackward0>)


In [44]:
def DQC_calculation_tensors(system, a, p):
    atomzs, atomposs = dqc.parse_moldesc(system)
    mol = dqc.Mol((atomzs, atomposs), basis="6-31G")
    
    myxc = MyLDAX(a, p)

    qc = dqc.KS(mol, xc=myxc).run()
    ene = qc.energy()  # calculate the energy
    return ene

def DQC_calculation_values(system, a_value, p_value):
    a = torch.nn.Parameter(torch.tensor(a_value, dtype=torch.double))
    p = torch.nn.Parameter(torch.tensor(p_value, dtype=torch.double))
    return DQC_calculation_tensors(system, a, p)

def finite_difference(system, a_value, p_value, coeff):
    y = DQC_calculation_values(system, a_value, p_value)
    da = a_value * coeff
    dp = p_value * coeff
    y_plus_dy_a = DQC_calculation_values(system, a_value+da, p_value)
    y_plus_dy_p = DQC_calculation_values(system, a_value, p_value+dp)
    dy_wrt_da = (y_plus_dy_a - y) / da
    dy_wrt_dp = (y_plus_dy_p - y) / dp
    return (dy_wrt_da, dy_wrt_dp)

finite_difference(water, 1.0, 2.0, 0.000001)

(tensor(9.9156, dtype=torch.float64, grad_fn=<DivBackward0>),
 tensor(19.0426, dtype=torch.float64, grad_fn=<DivBackward0>))

In [39]:
grad_a, grad_p = torch.autograd.grad(ene, (a_par, p_par))
print(grad_a, grad_p)

tensor(9.9156, dtype=torch.float64) tensor(19.0426, dtype=torch.float64)
